In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter



In [3]:
loader = TextLoader("sample.txt")
data = loader.load()
data

[Document(metadata={'source': 'sample.txt'}, page_content='In Germany, on the night of November 9, 1938, a quiet dread filled the air. The streets began to fill with the sounds of shattering glass, fire, and screams. That night, and into the next day, nearly 7,500 Jewish-owned businesses were smashed, hundreds of synagogues were burned, and thousands of Jewish men were arrested and sent to concentration camps.\n\nWhat triggered this violent outbreak?\n\nA 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.\n\nThe event was deceptively called Kristallnacht, or "The Night of Broken Glass" â€” referencing the glass from all the shattered shop windows. But what really broke that night wasn\'t just glass: it was the last illusion that German Jews could safely live under Nazi rule.\n\nAfter Kristallnacht, 

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits=text_splitter.split_documents(data)

In [5]:
embedding = OllamaEmbeddings(model="gemma:2b")
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
vectordb

C:\Users\namde\AppData\Local\Temp\ipykernel_8264\91020916.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="gemma:2b")


In [6]:
query = "What was the main motive of Herschel?"
docs = vectordb.similarity_search(query)
docs[0].page_content

'A 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.'

In [7]:
vectordb = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory="./chroma_db")


In [8]:
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embedding)
docs = db2.similarity_search(query)
print(docs[0].page_content)

A 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.


In [9]:
retriever = vectordb.as_retriever()
retriever.invoke(query)[0].page_content

'A 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.'